In [73]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from functools import reduce


"""
df
df = df_veg.merge(df_anim, left_on='Pays', right_on='Pays')

"""




names = ['Code Domaine', 'Domaine', 'Code Pays', 'Pays', 'Code Élément',
         'Élément', 'Code Produit', 'Produit', 'Code Année', 'Année', 'Unité',
         'Valeur', 'Symbole', 'Description du Symbole']


fn_population = 'fao_2013/FAOSTAT_2013_population.csv'
fn_animal = 'fao_2013/FAOSTAT_2013_animal.csv'
fn_vegetal = 'fao_2013/FAOSTAT_2013_vegetal.csv'

animaux = pd.read_csv(fn_animal, names=names, header=0)
vegetaux = pd.read_csv(fn_vegetal, names=names, header=0)
df_pop = pd.read_csv(fn_population, names=names, header=0)




# Ajout d'une colonne 'Origine' dans animaux et vegetaux
animaux['Origine'] = 'Animale'
vegetaux['Origine'] = 'Végétale'

# Union de Végétaux et Animaux
aliments = pd.concat([vegetaux, animaux])

# Projection
aliments = aliments[[
    'Pays',
    'Code Pays',
    'Produit',
    'Code Produit',
    'Origine',
    'Élément',
    'Valeur',
    'Année'
]]

# Table pivot sur aliments
aliments = aliments.pivot_table('Valeur', index=['Code Pays','Pays','Produit','Code Produit','Année','Origine'], columns='Élément')

# Reset index
aliments.reset_index(inplace=True)

# Joindre population et aliments
aliments = pd.merge(aliments, df_pop[['Pays', 'Valeur']], left_on='Pays', right_on='Pays')

# Renommer les colonnes
aliments.rename(columns={
    'Aliments pour animaux' : 'alim_ani',
    'Autres utilisations (non alimentaire)' : 'autres_utilisations',
    'Semences' : 'semences',
    'Pertes' : 'pertes',
    'Nourriture' : 'nourriture',
    'Traitement' : 'transfo', # ?
    'Disponibilité intérieure' : 'dispo_int',
    'Disponibilité alimentaire (Kcal/personne/jour)' : 'dispo_alim_kcal_p_j',
    'Disponibilité de protéines en quantité (g/personne/jour)' : 'dispo_prot',
    'Disponibilité de matière grasse en quantité (g/personne/jour)' : 'dispo_mat_gr',
    'Value' : 'population'
}, inplace=True)

# Remplacer les Nan
aliments.replace(np.nan, 0, inplace=True)

# Affichage
print('Résultat : ', aliments.shape[1], 'colonnes, ', aliments.shape[0], 'lignes')
aliments.sample(3)

#Calcul de la disponibilité en kcal par an pour tous les habitants du pays 
aliments['Disponibilté National en kcal par an par population'] = aliments['Disponibilité alimentaire en quantité (kg/personne/an)']*aliments['Valeur'] 

#Disponibilité des protéines en kg par an pour tous les habitants du pays
aliments['Disponibilté proteines en kg par an par pays'] = aliments['dispo_prot']*1000*365




Résultat :  22 colonnes,  4764 lignes


In [74]:
#Clcule du ratio en énergie et le poids par an pour chaque pays
aliments['Ratio/Energie/Poids'] = ((aliments['dispo_alim_kcal_p_j'] * 365) / aliments['Disponibilité alimentaire en quantité (kg/personne/an)'])
aliments.replace(np.nan, 0, inplace=True)
aliments


,Code Pays,Pays,Produit,Code Produit,Année,Origine,alim_ani,Autres Utilisations,dispo_alim_kcal_p_j,Disponibilité alimentaire en quantité (kg/personne/an),...,nourriture,pertes,Production,semences,transfo,Variation de stock,Valeur,Disponibilté National en kcal par an par population,Disponibilté proteines en kg par an par pays,Ratio/Energie/Poids
0,2,Afghanistan,Abats Comestible,2736,2013,Animale,0.0,0.0,5.0,1.72,...,53.0,0.0,53.0,0.0,0.0,0.0,30552,52549.44,281050.0,1061.046512
1,2,Afghanistan,"Agrumes, Autres",2614,2013,Végétale,0.0,0.0,1.0,1.29,...,39.0,2.0,3.0,0.0,0.0,0.0,30552,39412.08,7300.0,282.945736
2,2,Afghanistan,Aliments pour enfants,2680,2013,Végétale,0.0,0.0,1.0,0.06,...,2.0,0.0,0.0,0.0,0.0,0.0,30552,1833.12,10950.0,6083.333333
3,2,Afghanistan,Ananas,2618,2013,Végétale,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,30552,0.00,0.0,0.000000
4,2,Afghanistan,Bananes,2615,2013,Végétale,0.0,0.0,4.0,2.70,...,82.0,0.0,0.0,0.0,0.0,0.0,30552,82490.40,18250.0,540.740741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4759,256,Luxembourg,Viande de Suides,2733,2013,Animale,0.0,0.0,328.0,43.58,...,23.0,0.0,11.0,0.0,0.0,0.0,530,23097.40,4398250.0,2747.131712
4760,256,Luxembourg,Viande de Volailles,2734,2013,Animale,0.0,0.0,75.0,21.37,...,11.0,0.0,2.0,0.0,0.0,0.0,530,11326.10,2792250.0,1281.001404
4761,256,Luxembourg,"Viande, Autre",2735,2013,Animale,0.0,0.0,7.0,2.02,...,1.0,0.0,0.0,0.0,0.0,0.0,530,1070.60,346750.0,1264.851485
4762,256,Luxembourg,Vin,2655,2013,Végétale,0.0,0.0,100.0,51.41,...,27.0,0.0,10.0,0.0,0.0,0.0,530,27247.30,3650.0,709.978603


In [75]:
# question 5 : 5 aliments les plus calorique
df_max_cal = aliments[['Produit', 'Ratio/Energie/Poids']]
df_max_cal = df_max_cal.sort_values('Ratio/Energie/Poids', ascending=False)
df_max_cal.replace([np.inf, -np.inf], np.nan, inplace=True)
df_max_cal = df_max_cal.dropna(subset=['Ratio/Energie/Poids'])
df_max_cal.head(5)

,Produit,Ratio/Energie/Poids
1703,Huil Plantes Oleif Autr,18250.0
1540,Huile d'Olive,18250.0
725,Huile de Tournesol,18250.0
3402,Huile d'Olive,18250.0
3507,Huiles de Foie de Poisso,18250.0


In [76]:
# question 5 : 5 aliments les plus riches en protéines
df_max_prot = aliments[['Produit', 'Disponibilté proteines en kg par an par pays']]
df_max_prot = df_max_prot.sort_values('Disponibilté proteines en kg par an par pays', ascending=False)
df_max_prot.replace([np.inf, -np.inf], np.nan, inplace=True)
df_max_prot = df_max_prot.dropna(subset=['Disponibilté proteines en kg par an par pays'])
df_max_prot.head(5)

,Produit,Disponibilté proteines en kg par an par pays
2618,Poissons Pelagiques,20064050.0
7,Blé,13472150.0
2339,Maïs,13256800.0
1957,Blé,12402700.0
307,Riz (Eq Blanchi),11731100.0


In [77]:
# question 6 : disponibilité mondiale en millier de tonne (masse)
# poids moyen d'un être humain : 62 kg (masse )
# besoin calorique = metabolisme de base * 1.37 ~ 2000 kcal (énergie)
# masse moyenne théorique de population mondiale = 62 * pop_tot = 6997326000 * 62 = 433834212000 kg
df_dispo_mond = aliments[['Produit','Origine','dispo_int','Disponibilté proteines en kg par an par pays','Disponibilté National en kcal par an par population','Ratio/Energie/Poids', 'pertes', 'alim_ani']]
df_dispo_mond = df_dispo_mond[df_dispo_mond.Origine == 'Végétale']
df_dispo_mond['dispo_int_kcal'] = (10**5 * df_dispo_mond['dispo_int']) * df_dispo_mond['Ratio/Energie/Poids']
df_dispo_mond.replace([np.inf, -np.inf], np.nan, inplace=True)
df_dispo_mond = df_dispo_mond.dropna(subset=['dispo_int_kcal'])
print(df_dispo_mond)
# Question 7
pop_tot = 6997326000 
sum_dispo = df_dispo_mond['dispo_int_kcal'].sum()
sum_dispo_prot = df_dispo_mond['Disponibilté proteines en kg par an par pays'].sum()
food_needs = sum_dispo / (2000*365)
prot_needs = sum_dispo_prot / (62*365)
print(f"disponibilité végétale mondiale = {sum_dispo} kcal")
print(f"combien des personne peuvent être nourris : {food_needs / 10**6} millions de personnnes")
print(f"Pourcentage de la population concernée : {(food_needs / pop_tot)*100} %")
print(f"disponibilité des protéines végétales : {sum_dispo_prot} kg")
print(f"proportion des protéines couverts par les prétéines végétales : {prot_needs} personnes")
print(f"Pourcentage de la population concernée : {(prot_needs / pop_tot)*100} %")

                    Produit   Origine  dispo_int  \
1           Agrumes, Autres  Végétale       41.0   
2     Aliments pour enfants  Végétale        2.0   
3                    Ananas  Végétale        0.0   
4                   Bananes  Végétale       82.0   
6                     Bière  Végétale        3.0   
...                     ...       ...        ...   
4754                 Sésame  Végétale        0.0   
4755                    Thé  Végétale        0.0   
4756                Tomates  Végétale       12.0   
4762                    Vin  Végétale       27.0   
4763         Épices, Autres  Végétale        1.0   

      Disponibilté proteines en kg par an par pays  \
1                                           7300.0   
2                                          10950.0   
3                                              0.0   
4                                          18250.0   
6                                              0.0   
...                                            ... 

In [78]:
# Question 8 : Potentiel alimentaire des végétaux
# Résultat : aucun effet 
sum_pertes = df_dispo_mond['pertes'].sum()
sum_alim_anim = df_dispo_mond['alim_ani'].sum()
sum_dispo_anim_pertes = sum_dispo + sum_pertes + sum_alim_anim
sum_dispo_prot_anim_pertes = sum_dispo_prot + sum_pertes + sum_alim_anim
print(sum_dispo, sum_pertes, sum_alim_anim)
food_needs = sum_dispo_anim_pertes / (2000*365)
prot_needs = sum_dispo_prot_anim_pertes / (62*365)
print(f"disponibilité végétale mondiale = {sum_dispo} kcal")
print(f"combien des personne peuvent être nourris : {food_needs / 10**6} millions de personnnes")
print(f"Pourcentage de la population concernée : {(food_needs / pop_tot)*100} %")
print(f"disponibilité des protéines végétales : {sum_dispo_prot} kg")
print(f"proportion des protéines couverts par les prétéines végétales : {prot_needs} personnes")
print(f"Pourcentage de la population concernée : {(prot_needs / pop_tot)*100} %")

176388481579773.47 64322.0 164371.0
disponibilité végétale mondiale = 176388481579773.47 kcal
combien des personne peuvent être nourris : 241.62805727187185 millions de personnnes
Pourcentage de la population concernée : 3.453148492322237 %
disponibilité des protéines végétales : 866371300.0 kg
proportion des protéines couverts par les prétéines végétales : 38294.29929297393 personnes
Pourcentage de la population concernée : 0.0005472704757928089 %


In [79]:
# Question 9 
print(df_dispo_mond) 
dispo_alim_mond = df_dispo_mond['Disponibilté National en kcal par an par population'].sum()
dispo_prot_mond = df_dispo_mond['Disponibilté proteines en kg par an par pays'].sum()
print(f"dispo alimentaire mondiale totale : {dispo_alim_mond} kcal par an") 
print(f"dispo protéique alimentaire mondiale : {dispo_prot_mond} kg par an")
print(f"Pourcentage de la population concernée : {(dispo_alim_mond / pop_tot)*100} %")
print(f"Pourcentage de la population concernée : {(dispo_prot_mond / pop_tot)*100} %")

                    Produit   Origine  dispo_int  \
1           Agrumes, Autres  Végétale       41.0   
2     Aliments pour enfants  Végétale        2.0   
3                    Ananas  Végétale        0.0   
4                   Bananes  Végétale       82.0   
6                     Bière  Végétale        3.0   
...                     ...       ...        ...   
4754                 Sésame  Végétale        0.0   
4755                    Thé  Végétale        0.0   
4756                Tomates  Végétale       12.0   
4762                    Vin  Végétale       27.0   
4763         Épices, Autres  Végétale        1.0   

      Disponibilté proteines en kg par an par pays  \
1                                           7300.0   
2                                          10950.0   
3                                              0.0   
4                                          18250.0   
6                                              0.0   
...                                            ... 

In [84]:
# Question 10 : sous nutrition
df_malnutrition = pd.read_csv('fao_2013/FAOSTAT_2013_sous_alimentation.csv')
sum_malnutrition = df_malnutrition['Valeur'].sum()
print(f"{100 * (sum_malnutrition * 10**6 / pop_tot)} % de la population mondiale souffre de sous nutrition")

12.533359171775047 % de la population mondiale souffre de sous nutrition


In [ ]:
# Question 11 : Etude sur les céréales